# Viele Dateien

**Inhalt:** Massenverarbeitung von gescrapten Zeitreihen

**Nötige Skills:** Daten explorieren, Time+Date Basics

**Lernziele:**
- Pandas in Kombination mit Scraping
- Öffnen und zusammenfügen von vielen Dateien (Glob)
- Umstrukturierung von Dataframes (Pivot)
- Plotting Level 4 (Small Multiples)

## Das Beispiel

Wir interessieren uns in diesem Notebook für Krypto-Coins.

Die Webseite https://coinmarketcap.com/ führt Marktdaten zu den hundert wichtigsten Coins auf.

Mit einem einfachen Scraper werden wir diese Daten beschaffen und rudimentär analysieren.

Der Pfad zum Projektordner heisst `dataprojects/Krypto/`

## Vorbereitung

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import re

In [ ]:
import glob

In [ ]:
%matplotlib inline

## Scraper

In [ ]:
path = 'dataprojects/Krypto/'

### Liste von allen Kryptowährungen

Zuerst kucken wir auf der Seite, welches die 100 grössten Kryptowährungen sind, und laden uns Namen und Links derselbigen.

In [ ]:
base_url = 'https://coinmarketcap.com/'

In [ ]:
response = requests.get(base_url)
doc = BeautifulSoup(response.text, "html.parser")

In [ ]:
currencies = doc.find_all('a', class_='currency-name-container link-secondary')

In [ ]:
currencies[0]

In [ ]:
len(currencies)

In [ ]:
currency_list = []

In [ ]:
for currency in currencies:
    this_currency = {}
    this_currency['name'] = currency.text
    this_currency['link'] = currency['href']
    currency_list.append(this_currency)

In [ ]:
df_currencies = pd.DataFrame(currency_list)

In [ ]:
df_currencies.head(2)

In [ ]:
df_currencies['link'] = df_currencies['link'].str.extract('/currencies/(.+)/')

In [ ]:
df_currencies.head(2)

In [ ]:
df_currencies.to_csv(path + 'currencies.csv', index=False)

### Daten von den einzelnen Währungen

Zuerst testen wir mit einer Probewährung aus, wie wir an die Informationen kommen.

In [ ]:
base_url = 'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20171015&end=20181015'

In [ ]:
response = requests.get(base_url)
doc = BeautifulSoup(response.text, "html.parser")

In [ ]:
days = doc.find_all('tr', class_='text-right')

In [ ]:
days_list = []

In [ ]:
cells = days[0].find_all('td')

In [ ]:
cells

In [ ]:
this_day = {}

In [ ]:
this_day['date'] = cells[0].text
this_day['open'] = cells[1].text
this_day['high'] = cells[2].text
this_day['low'] = cells[3].text
this_day['close'] = cells[4].text
this_day['volume'] = cells[5].text
this_day['marketcap'] = cells[6].text

In [ ]:
this_day

In [ ]:
for day in days:
    this_day = {}
    cells = day.find_all('td')
    this_day['date'] = cells[0].text
    this_day['open'] = cells[1].text
    this_day['high'] = cells[2].text
    this_day['low'] = cells[3].text
    this_day['close'] = cells[4].text
    this_day['volume'] = cells[5].text
    this_day['marketcap'] = cells[6].text
    days_list.append(this_day)

In [ ]:
df = pd.DataFrame(days_list)

In [ ]:
df.head(2)

Nun wenden wir den Scraper auf alle Währungen an

In [ ]:
df_currencies = pd.read_csv(path + 'currencies.csv')

In [ ]:
df_currencies.head(2)

In [ ]:
len(df_currencies)

In [ ]:
currencies = df_currencies.to_dict(orient='records')

In [ ]:
url_start = 'https://coinmarketcap.com/currencies/'
url_end = '/historical-data/?start=20171015&end=20181015'

In [ ]:
for currency in currencies:
    print ('working on: ' + currency['name'])
    
    url = url_start + currency['link'] + url_end
    response = requests.get(url)
    doc = BeautifulSoup(response.text, "html.parser")
    
    days = doc.find_all('tr', class_='text-right')
    days_list = []
    
    this_day = {}
    for day in days:
        this_day = {}
        cells = day.find_all('td')
        this_day['date'] = cells[0].text
        this_day['open'] = cells[1].text
        this_day['high'] = cells[2].text
        this_day['low'] = cells[3].text
        this_day['close'] = cells[4].text
        this_day['volume'] = cells[5].text
        this_day['marketcap'] = cells[6].text
        days_list.append(this_day)
        
    df = pd.DataFrame(days_list)
    filename = currency['name'] + '.csv'
    df.to_csv(path + 'data/' + filename, index=False)
    
print('Done')

Am Ende haben wir eine Liste von Dateien: Zu jeder Kryptowährung existiert eine Tabelle mit den Marktdaten über den definierten Zeitraum.

Die Daten sind im Unterordner `data/` abgelegt.

## Daten analysieren

### Einlesen

Wir starten damit, dass wir das Verzeichnis durchsuchen, in dem alle Kryptowährungs-Daten abgelegt sind.

Dazu benutzen wir `glob`, ein praktisches Tool aus der Standard Library: https://docs.python.org/3/library/glob.html

In [ ]:
filenames = glob.glob(path + 'data/*.csv')

In [ ]:
len(filenames)

In [ ]:
filenames[0:2]

Mit Glob haben wir nun eine Liste mit den Dateinamen erstellt.

Nun lesen wir jede einzelne Datei aus der Liste ein.

So, dass wir als Ergebnis eine Liste von Dataframes erhalten.

In [ ]:
dfs = []

In [ ]:
dfs = [pd.read_csv(filename) for filename in filenames]

In [ ]:
dfs[0].head(2)

Die einzelnen Dataframes in der Liste enthalten die Marktdaten. Doch sie enthalten selbst keine Information darüber, zu welcher Kryptowährung die Daten gehören. Wir führen zu dem Zweck in jedes Dataframe noch eine zusätzliche Spalte hinzu mit dem Namen der Währung. 

In [ ]:
for df, filename in zip(dfs, filenames):
    df['currency'] = filename
    df['currency'] = df['currency'].str.extract('/data/(.+).csv')

In [ ]:
dfs[0].head(2)

Nun fügen wir alle Dataframes zu einem einzigen, sehr langen Dataframe zusammen.

Dazu benutzen wir die Funktion `pd.concat()`: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html

In [ ]:
df_all = pd.concat(dfs, ignore_index=True)

In [ ]:
df_all.shape

In [ ]:
df_all.head(2)

In [ ]:
df_all.tail(2)

In [ ]:
df_all.dtypes

Wir haben nun ein ellenlanges Dataframe. What next?

### Arrangieren

Das hängt davon ab, was wir mit den Daten genau tun wollen.

Eine Option wäre: die verschiedenen Währungen miteinander zu vergleichen. Und zwar anhand der Schlusskurse.

Dazu müssen wir das Dataframe leicht umstellen, mit `pivot()`: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html

In [ ]:
df_pivoted = df_all.pivot(index='date', columns='currency', values='close')

In [ ]:
df_pivoted.shape

In [ ]:
df_pivoted.head(2)

In [ ]:
df_pivoted.tail(2)

In [ ]:
df_pivoted.rename_axis(None, inplace=True)

Nun verfügen wir über einen Index, bei dem eine Zeile jeweils einem einzigartigen Zeitpunkt entspricht.

Um damit zu arbeiten, verwandeln wir den Text in der Indexspalte in ein echtes Datum des Typs datetime.

In [ ]:
df_pivoted.index = pd.to_datetime(df_pivoted.index, format="%b %d, %Y")

In [ ]:
df_pivoted.sort_index(inplace=True)

In [ ]:
df_pivoted.head(2)

Wir haben nun ein sauber formatiertes Dataframe. Mit hundert Spalten, die für jede Kryptowährung, sofern sie zum betreffenden Zeitpunkt existierte, einen Handelskurs enthält.

Die nächste Frage ist: Wie vergleichen wir diese Kurse? Was sagt es aus, wenn eine Währung an einem bestimmten Tag zu 0,1976 USD gehandelt wurde und eine andere zu 18,66 USD?

### Vergleichbarkeit herstellen

Diverse Dinge würden sich hier anbieten:
- zB `pct_change()` um die Veränderungen in den Kursen zu analysieren
- oder eine indexierte Zeitreihe, die an einem bestimmten Tag bei 100 beginnt

Wir wählen die zweite Variante. Und speichern dazu die erste Zeile separat ab.

In [ ]:
row_0 = df_pivoted.iloc[0]

Dann teilen wir jede einzelne Zeile im Dataframe durch die erste Zeile. Und speichern als neues DF ab.

In [ ]:
df_pivoted_100 = df_pivoted.apply(lambda row: row / row_0 * 100, axis=1)

Das neue Dataframe ist nun indexiert auf 100. Alle Währungen starten am gleichen Punkt...

In [ ]:
df_pivoted_100.head(5)

In [ ]:
df_pivoted_100.tail(1)

... und enden an einem bestimmten Punkt. Anhand dieses Punktes können wir die relative Entwicklung ablesen.

In [ ]:
s_last = df_pivoted_100.iloc[-1]

Welche zehn Kryptowährungen am meisten Wert zugelegt haben...

In [ ]:
s_last.sort_values(ascending=False).head(10)

... und welche am meisten Wert verloren haben.

In [ ]:
s_last.sort_values(ascending=False, na_position='first').tail(10)

Und so sieht die Performance aller Währungen aus:

In [ ]:
df_pivoted_100.plot(figsize=(10,6), legend=False)

Wow, das sind ziemlich viele Linien!

# Plotting Level 4

Wie wir diesen Chart etwas auseinandernehmen können, lernen wir hier.

Eine Gelegenheit, zu sehen, wie man die matplotlib-Funktionen direkt benutzen kann.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

Und eine neue Art kennenlernen, wie man einen Plot erstellt.

### Ein Plot

Starten wir zuerst mal mit einem Plots: Bitcoin.

Wir müssen uns dazu zuerst zwei Dinge basteln:
1. Eine "figure", also eine Abbildung
1. Einen "subplot", also der Plot selbst

In [ ]:
# Wir erstellen beide Dinge in einem Atemzug
fig, ax = plt.subplots(figsize=(10,6))

# Und füllen den Plot jetzt mit Inhalt:
df_pivoted_100['Bitcoin'].plot(title="Bitcoin", ax=ax)

### Zwei Plots

Als nächstes Plotten wir zwei Währungen auf derselben Figure: Bitcoin und Ethereum.

Wir müssen uns dazu erneut zwei Dinge basteln:
1. Eine "figure", also eine Abbildung
1. Diverse "subplots" für die jeweiligen Währungen

Dazu formatieren wir jetzt die x-Achse etwas speziell.

In [ ]:
# Zuerst kreieren wir nur die Figure
fig = plt.figure(figsize=(12,3))

# Danach die einzelnen Subplots
ax1 = fig.add_subplot(1, 2, 1) # total 1 Zeile, total 2 Spalten, Subplot Nr. 1
ax2 = fig.add_subplot(1, 2, 2) # total 1 Zeile, total 2 Spalten, Subplot Nr. 2

# Und schliesslich füllen wir die Subplots mit Inhalt
df_pivoted_100['Bitcoin'].plot(title="Bitcoin", ax=ax1)
df_pivoted_100['Ethereum'].plot(title="Ethereum", ax=ax2)

# Hier formatieren wir die x-Achse für Plot 1
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m'))
ax1.xaxis.set_minor_locator(ticker.NullLocator())

# Hier formatieren wir die x-Achse für Plot 2
ax2.xaxis.set_major_locator(mdates.MonthLocator())
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%m'))
ax2.xaxis.set_minor_locator(ticker.NullLocator())

Einige Angaben dazu, wie man Zeitachsen formatieren kann, gibt es hier:
- TickLocators: https://matplotlib.org/examples/ticks_and_spines/tick-locators.html
- TickFormatters: https://matplotlib.org/gallery/ticks_and_spines/tick-formatters.html

### Sehr viele Plots

Nun plotten wir sämtliche Währungen auf einmal. Wie viele sind es?

In [ ]:
anzahl_charts = s_last.notnull().sum()
anzahl_charts

Wir sortieren unsere Liste der Währungen etwas:

In [ ]:
sortierte_waehrungen = s_last[s_last.notnull()].sort_values(ascending=False)
sortierte_waehrungen.head(2)

Und wiederholen dann wiederum dasselbe Vorgehen wie vorher.

In [ ]:
# Eine Abbildung, die gross genug ist
fig = plt.figure(figsize=(15,22))

# Und nun, für jede einzelne Währung:
for i, waehrung in enumerate(sortierte_waehrungen.index):
    
    # einen Subplot kreieren ...
    ax = fig.add_subplot(11, 6, i + 1)

    # ... und mit Inhalt füllen
    df_pivoted_100[waehrung].plot(title=waehrung, ax=ax)
    
    # Auf Ticks verzichten wir hier ganz
    ax.xaxis.set_major_locator(ticker.NullLocator())
    ax.xaxis.set_minor_locator(ticker.NullLocator())

Falls wir zusätzlich noch wollen, dass jeder Plot dieselbe y-Achse hat:

In [ ]:
# Eine Abbildung, die gross genug ist
fig = plt.figure(figsize=(15,22))

# Und nun, für jede einzelne Währung:
for i, waehrung in enumerate(sortierte_waehrungen.index):
    
    # einen Subplot kreieren ...
    ax = fig.add_subplot(11, 6, i + 1)

    # ... und mit Inhalt füllen
    df_pivoted_100[waehrung].plot(title=waehrung, ax=ax)
    
    # Auf Ticks verzichten wir hier ganz
    ax.xaxis.set_major_locator(ticker.NullLocator())
    ax.xaxis.set_minor_locator(ticker.NullLocator())
    
    # Hier setzen wir eine einheitliche y-Achse (und schalten sie aus)
    ax.set_ylim([0, 25000])
    ax.yaxis.set_major_locator(ticker.NullLocator())

### Aber es geht auch einfacher...

Ha! Nachdem wir nun alles Manuell zusammengebastelt haben, mit Matplotlib, hier die gute Nachricht:

*Wir können das mit wenigen Codezeilen auch direkt aus der Pandas-Plot()-Funktion haben :-)*

In [ ]:
axes = df_pivoted_100[sortierte_waehrungen.index].plot(subplots=True,layout=(11, 6), sharey=True, figsize=(15,22))

axes[0,0].xaxis.set_major_locator(ticker.NullLocator())
axes[0,0].xaxis.set_minor_locator(ticker.NullLocator())

# Übung

Hier schauen wir uns nicht mehr die Handelskurse, sondern die Handelsvolumen an! Also: Wie viel von den einzelnen Kryptowährungen an einem bestimmten Tag gekfauft und verkauft wurde (gemessen in USD).

Schauen Sie sich nochmals das Dataframe `df_all` an, das wir im Verlauf des Notebooks erstellt haben - es enthält alle Informationen, die wir brauchen, ist aber noch relativ unstrukturiert.

Welche Spalte interessiert uns? Müssen wir noch etwas daran machen?

### Daten arrangieren

Unternehmen Sie die nötigen Schritte, um mit der Spalte arbeiten zu können. Sie sollten am Ende eine Spalte haben, die nicht mehr als Object, sondern als Float formatiert ist.

Tipp: Speichern Sie alle Modifikationen in einer neuen Spalte ab, damit das Original unverändert bleibt.

Nun wollen wir die Daten umgliedern:
- Für jedes Datum wollen wir eine Zeile
- Für jede Kryptowährung eine Spalte
- Wir interessieren uns für die Handelsvolumen

Formatieren Sie die Werte in der Index-Spalte als Datetime-Objekte und sortieren Sie das Dataframe nach Datum.

### Analyse

Wir machen in dieser Sektion einige einfache Auswertungen und repetieren einige Befehle, u.a. aus dem Time Series Sheet.

**Top-10**: Welches waren, im Schnitt, die zehn meistgehandelten Währungen? Liste und Chart.

Welches waren die zehn Währungen, bei denen das Volumen in absoluten Zahlen am meisten geschwankt ist? (Standardabweichung)

Sieht so aus, als wären es dieselben zehn Währungen.

Können wir angeben, welche von ihnen relativ die grössten Schwankungen hatten, also im Vergleich zum Handelsvolumen?

**Bitcoin vs Ethereum**

Erstellen Sie einen Chart mit dem wöchentlichen Umsatztotal von Bitcoin und Ethereum!

In welchem der letzten 12 Monate wurde insgesamt am meisten mit Bitcoin gehandelt? Mit Ethereum?

Wie viel Bitcoin und Ethereum wird im Durchschnitt an den sieben Wochentagen gehandelt? Barchart.

**Small Multiples**: Hier erstellen wir einen Plot, ähnlich wie oben

Kreieren Sie zuerst eine Liste von Währungen:
- Alle Währungen, die am letzten Handelstag einen Eintrag haben
- Sortiert in absteigender Reihenfolge nach dem Handelsvolumen
- Wir wählen nur die zehn grössten aus

Und jetzt: Small Multiples plotten! Überlegen Sie sich:
- Wie viele Subplots braucht es, wie sollen sie angeordnet sein?
- Wie gross muss die Abbildung insgesamt sein?
- Was ist eine sinnvolle Einstellung für die Y-Achse?

(Sie können die Matplotlib-Funktionalität dafür nutzen oder direkt Pandas-plot()